# 🗣 Language Model N-Gram

It's my desire to explore the unknown and discover meaning in data. With the rise of GPT 3, I wanted to create my own language model on a library of over 70,000 eBooks.

Summarized Professor Daniel Jurafsky from Stanford University, language models attempt to capture the likelihood that a given sequence of words occur in a given collection of texts. Such a model, for example, could predict that the following sequence has a much higher probability of appearing in a text:
- *all of a sudden I notice three guys standing on the sidewalk*

than does this same set of words in a different order:

- *on guys all I of notice sidewalk three a sudden standing the*

As with all statistical models, the true data generating process is unknown to us, so all we can do is **estimate** the probabilities of sentences. For example, one might estimate the probability of a sentence as simply the product of the empirical probabilities (i.e., the number of times a word is observed in a dataset divided by the number of words in that dataset). 

$$P(\text{when I drink Coke I smile}) = P(\text{when}) \cdot P(\text{I}) \cdot P(\text{drink}) \cdot P(\text{Coke}) \cdot P(\text{I}) \cdot P(\text{smile})$$


I start off by building the simplest model that assigns probabilities to sentences and sequences of words, the **n-gram**. An n-gram is a sequence n-gram of n words: a 2-gram (called bigram) is a two-word sequence of words like “please turn”, “turn your”, or ”your homework”, and a 3-gram (a trigram) is a three-word sequence of words like “please turn your”, or “turn your homework”.

I use math to estimate the probability of the last word of an n-gram given the previous words, and also to assign probabilites to entire sequences.

## 📚 Import Libraries 

In [34]:
import pandas as pd
import numpy as np
import os
import re
import requests
import time

## 🪚 1) Preparing the Corpus

I'll use the `requests` module to download the "Plain Text UTF-8" text of a public domain book from [Project Gutenberg](https://www.gutenberg.org/) and prepare it for analysis in later questions. For instance, the book Beowulf's "Plain Text UTF-8" URL is [here](https://www.gutenberg.org/ebooks/16328.txt.utf-8), which can be accessed by clicking the "Plain Text UTF-8" link [here](https://www.gutenberg.org/ebooks/16328).

In [9]:

# Retrieves contents of eBook through HTTP request
def get_book(url):
    
    # HTTP request to Project Gurenberg
    text = requests.get(url).text
    
    # Locate title of Book
    title = re.findall(r'Title: ([A-Za-z ]+)', text)[0].upper()
    
    # Regex --> Located starting point of given boon
    pattern = r'\*{3} START OF (?:THE|THIS) PROJECT GUTENBERG EBOOK [\r\n \w]+ \*{3}((?s).*)\*{3} END OF (?:THE|THIS) PROJECT GUTENBERG EBOOK [\r\n \w]+ \*{3}'
    
    # Apply regex to the content
    content = re.findall(pattern, text)[0]
    
    # Replace Windows newline characters ('\r\n') with standard newline characters 
    return re.sub(r'\r\n', '\n', content)


### ✅ Testing on The Great Gatsby

In [29]:
# Great Gatsby URL provided in Plain Text UTF-8
great_gatsby = get_book("https://www.gutenberg.org/cache/epub/64317/pg64317.txt")

# First 2000 characters of The Great Gatsby
great_gatsby[:2000]

'\n        \n\n\t\t\t   The Great Gatsby\n\t\t\t\t  by\n\t\t\t F. Scott Fitzgerald\n\n\n                           Table of Contents\n\nI\nII\nIII\nIV\nV\nVI\nVII\nVIII\nIX\n\n\n                              Once again\n                                  to\n                                 Zelda\n\n  Then wear the gold hat, if that will move her;\n  If you can bounce high, bounce for her too,\n  Till she cry “Lover, gold-hatted, high-bouncing lover,\n  I must have you!”\n\n  Thomas Parke d’Invilliers\n\n\n                                  I\n\nIn my younger and more vulnerable years my father gave me some advice\nthat I’ve been turning over in my mind ever since.\n\n“Whenever you feel like criticizing anyone,” he told me, “just\nremember that all the people in this world haven’t had the advantages\nthat you’ve had.”\n\nHe didn’t say any more, but we’ve always been unusually communicative\nin a reserved way, and I understood that he meant a great deal more\nthan that. In consequence, I’

## 🪓 2) Tokenizing the Corpus

I **tokenize** the text by implementing the function `tokenize`, which takes in a string, `book_string`, and returns a **list of the tokens** (words, numbers, and all punctuation) in the book such that:

* The start of every paragraph is represented in the list with the single character `'\x02'` (standing for START).
* The end of every paragraph is represented in the list with the single character `'\x03'` (standing for STOP).
* Tokens include *no* whitespace.
* Two or more newlines count as a paragraph break, and whitespace (e.g. multiple newlines) between two paragraphs of text do not appear as tokens.
* All punctuation marks count as tokens, even if they are uncommon (e.g. `'@'`, `'+'`, and `'%'` are all valid tokens).

For example, consider the following excerpt. (The first sentence is at the end of a larger paragraph, and the second sentence is at the start of a longer paragraph.)
```
...
My phone's dead.

I didn't get your call!!
...
```
Tokenizes to:
```py
[...
'My', 'phone', "'", 's', 'dead', '.', '\x03', '\x02', 'I', 'didn', "'", 't', 'get', 'your', 'call', '!', '!'
...]
```

In [20]:
def tokenize(book_string):
    
    # Remove any leading or trailing spaces from the original 
    book_string = '\x02'+book_string.strip()+'\x03'
    
    # Replace the beginning of two or more consecutive newlines 
    book_string = re.sub('^\n{2,}', '\x02', book_string)
    
    # Replace the end of two or more consecutive newlines 
    book_string = re.sub('\n{2,}$', '\x03', book_string)
    
    # Replaces any other occurrence of two or more consecutive newlines 
    book_string = re.sub('\n{2,}', '\x03\x02', book_string)
    
    # Matches any alpha or non-alpha characters
    pattern = r'[A-Za-z]+|[^\s\d\w]|\x03|\x02'
    
    # Returns list containing individual tokens extracted from the book_string
    return re.findall(pattern, book_string)

### ✅ Testing: tokenize() on The Great Gatsby

In [38]:
# Tokenize the Great Gatsby
tokenized_gatsby = tokenize(great_gatsby)

# First 200 tokens
np.array(tokenized_gatsby)[:200]

array(['\x02', 'The', 'Great', 'Gatsby', 'by', 'F', '.', 'Scott',
       'Fitzgerald', '\x03', '\x02', 'Table', 'of', 'Contents', '\x03',
       '\x02', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX',
       '\x03', '\x02', 'Once', 'again', 'to', 'Zelda', '\x03', '\x02',
       'Then', 'wear', 'the', 'gold', 'hat', ',', 'if', 'that', 'will',
       'move', 'her', ';', 'If', 'you', 'can', 'bounce', 'high', ',',
       'bounce', 'for', 'her', 'too', ',', 'Till', 'she', 'cry', '“',
       'Lover', ',', 'gold', '-', 'hatted', ',', 'high', '-', 'bouncing',
       'lover', ',', 'I', 'must', 'have', 'you', '!', '”', '\x03', '\x02',
       'Thomas', 'Parke', 'd', '’', 'Invilliers', '\x03', '\x02', 'I',
       '\x03', '\x02', 'In', 'my', 'younger', 'and', 'more', 'vulnerable',
       'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that',
       'I', '’', 've', 'been', 'turning', 'over', 'in', 'my', 'mind',
       'ever', 'since', '.', '\x03', '\x02', '“', 'Whenever', 'you',
  

## Creating Baseline Language Models 📕

Sentences are built from tokens, and the likelihood that a token occurs where it does depends on the tokens before it. This points to using **conditional probability** to compute $P(w)$. That is, we can write:

$$
P(w) = P(w_1,\ldots,w_n) = P(w_1) \cdot P(w_2|w_1) \cdot P(w_3|w_1,w_2) \cdot\ldots\cdot P(w_n|w_1,\ldots,w_{n-1})
$$  
Using **chain rule** for probabilities.

**Example:** 

<center><code>'when I drink Coke I smile'</code></center>
    
The probability that it occurs, according the the chain rule, is

$$
P(\text{when}) \cdot P(\text{I | when}) \cdot P(\text{drink | when I})\cdot P(\text{Coke | when I drink}) \cdot P(\text{I | when I drink Coke}) \cdot P(\text{smile | when I drink Coke I})
$$

That is, the probability that the sentence occurs is the product of the probability that each subsequent token follows the tokens that came before. For example, the probability $P(\text{Coke | when I drink})$ is likely pretty high, as Coke is something that you drink. The probability $P(\text{pizza | when I drink})$ is likely low, because pizza is not something that you drink.



## 🎲 Creating a Unifrom Model

A uniform language model is one in which each **unique** token is equally likely to appear in any position, unconditional of any other information. In other words, in a uniform language model, the probability assigned to each token is **1 over the total number of unique tokens in the corpus**.


```py
>>> corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
>>> tokenize(corpus)
['\x02', 'when', 'I', 'eat', 'pizza', ',', 'I', 'smile', ',', 'but', 'when', 'I', 'drink', 'Coke', ',', 'my', 'stomach', 'hurts', '\x03']
```

The example corpus above has 14 **unique** tokens. This means that I'd have $P(\text{\x02}) = \frac{1}{14}$, $P(\text{when}) = \frac{1}{14}$, and so on. Specifically, in this example, **the Series that `train` returns should contain the following values**:

| Token | Probability |
| --- | --- |
| `'\x02'` | $\frac{1}{14}$ |
| `'when'` | $\frac{1}{14}$ |
| `'I'` | $\frac{1}{14}$ |
| `'eat'` | $\frac{1}{14}$ |
| `'pizza'` | $\frac{1}{14}$ |
| `','` | $\frac{1}{14}$ |
| `'smile'` | $\frac{1}{14}$ |
| `'but'` | $\frac{1}{14}$ |
| `'drink'` | $\frac{1}{14}$ |
| `'Coke'` | $\frac{1}{14}$ |
| `'my'` | $\frac{1}{14}$ |
| `'stomach'` | $\frac{1}{14}$ |
| `'hurts'` | $\frac{1}{14}$ |
| `'\x03'` | $\frac{1}{14}$ |

#### Unifrom Class:

* The `__init__` constructor: when you instantiate an LM object, I pass in the "training corpus" on which my model will be trained. The `train` method uses that data to create a model which is saved in the `mdl` attribute. 
* The `train` method takes in a list of tokens and outputs a language model. **This language model is represented as a `Series`, whose index consists of tokens and whose values are the probabilities that the tokens occur.** 
* The `probability` method takes in a sequence of tokens and returns the probability that this sequence occurs under the language model.
* The `sample` method takes in a positive integer `M` and generates a string made up of `M` tokens using the language model. **This method generates random sentences!**

In [51]:
class UniformLM(object):

    def __init__(self, tokens):
        # Constructor method
        # Initializes the class instance and trains the language model.
        # 'tokens' is a list of words used to train the language model.
        self.mdl = self.train(tokens)
        
    def train(self, tokens):
        # Training method
        # Creates a uniform language model from the provided list of tokens.
        # 'tokens' is a list of words used to build the vocabulary of the model.
        unique_tokens = pd.Series(tokens).unique()
        # Calculate the uniform probability for each unique token and create a pandas Series.
        return pd.Series(np.full(len(unique_tokens), 1/len(unique_tokens)), index=unique_tokens)
    
    def probability(self, words):
        # Probability calculation method
        # Calculates the probability of a sequence of words given the trained language model.
        try:
            # Convert 'words' to a list to ensure we can access elements by index.
            words = list(words)
            # Calculate the probability of the sequence by taking the product of individual word probabilities.
            prob = np.prod(self.mdl.loc[words])
        except KeyError:
            # If any word in the sequence is not present in the language model's vocabulary,
            # set the probability to 0 indicating that the sequence is not recognized.
            prob = 0
        return prob 
        
    def sample(self, M):
        # Sampling method
        # Generates a random sequence of 'M' words from the language model.
        # 'M' is the number of words to be sampled.
        return ' '.join(self.mdl.sample(M, replace=True).index)
        # Generate a random sample of 'M' words with replacement from the language model's vocabulary.
        # Join the sampled words into a string with spaces between them and return the string.


### ✅ Testing: UnifromML() on The Great Gatsby

In [63]:
tokens = tokenized_gatsby

# Initilize and train Unifrom model on given tokens
unif = UniformLM(tokens)

# Generates a random sequence of 100 words from the language model.
unif.sample(100)

'drum argue waiter Haven His Probably eyebrow dates Schraeders Montana powdered representing emergency brooded Backhyssons bathrooms support betting composed Platonic rumour Taking pulpless Flushing dog eager visitors wheel upright desolate afford despairing painting excellence bles getting pomp infantry scalloped destiny riding frequently preyed floating mistake facet February Americans ran tonic Tom wildly tumult glamour rounds orange Never Another event complained slump De breath gambler Be fluttered realize ridges judging entangled paralysed bles smart ceiling objects rubber grey firmly trees Inside stifling Eckleburg tearing Eggers permanently laudable events Webster eyes explicable each retribution star tactlessly fare meanwhile can except rapidly fix'

## 🪇 Creating a Uni-gram Model

A unigram language model is one in which the **probability assigned to a token is equal to the proportion of tokens in the corpus that are equal to said token**. That is, the probability distribution associated with a unigram language model is just the empirical distribution of tokens in the corpus. 

Let's understand how probabilities are assigned to tokens using our example corpus from before.

```py
>>> corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
>>> tokenize(corpus)
['\x02', 'when', 'I', 'eat', 'pizza', ',', 'I', 'smile', ',', 'but', 'when', 'I', 'drink', 'Coke', ',', 'my', 'stomach', 'hurts', '\x03']
```

Here, there are 19 total tokens. 3 of them are equal to `'I'`, so $P(\text{I}) = \frac{3}{19}$. Here, the Series that `train` returns should contain the following values:

| Token | Probability |
| --- | --- |
| `'\x02'` | $\frac{1}{19}$ |
| `'when'` | $\frac{2}{19}$ |
| `'I'` | $\frac{3}{19}$ |
| `'eat'` | $\frac{1}{19}$ |
| `'pizza'` | $\frac{1}{19}$ |
| `','` | $\frac{3}{19}$ |
| `'smile'` | $\frac{1}{19}$ |
| `'but'` | $\frac{1}{19}$ |
| `'drink'` | $\frac{1}{19}$ |
| `'Coke'` | $\frac{1}{19}$ |
| `'my'` | $\frac{1}{19}$ |
| `'stomach'` | $\frac{1}{19}$ |
| `'hurts'` | $\frac{1}{19}$ |
| `'\x03'` | $\frac{1}{19}$ |

As before, the `probability` method should take in a tuple and return its probability, using the probabilities stored in `mdl`. For instance, suppose the input tuple is `('when', 'I', 'drink', 'Coke', 'I', 'smile')`. Then,

$$P(\text{when I drink Coke I smile}) = P(\text{when}) \cdot P(\text{I}) \cdot P(\text{drink}) \cdot P(\text{Coke}) \cdot P(\text{I}) \cdot P(\text{smile}) = \frac{2}{19} \cdot \frac{3}{19} \cdot \frac{1}{19} \cdot \frac{1}{19} \cdot \frac{3}{19} \cdot \frac{1}{19}$$

In [57]:
class UnigramLM(object):

    def __init__(self, tokens):
        # Constructor method
        # Initializes the class instance and trains the unigram language model.
        # 'tokens' is a list of words used to train the language model.
        self.mdl = self.train(tokens)
    
    def train(self, tokens):
        # Training method
        # Creates a unigram language model from the provided list of tokens.
        # 'tokens' is a list of words used to build the vocabulary of the model.
        tokens = pd.Series(tokens)
        # Convert 'tokens' to a pandas Series to perform value counting.
        return tokens.value_counts().apply(lambda x: x/len(tokens))
        # Calculate the frequency (probability) of each token and normalize it by dividing by the total number of tokens.
        # Create a pandas Series where the index represents each unique token and the values represent their probabilities.

    def probability(self, words):
        # Probability calculation method
        # Calculates the probability of a sequence of words given the trained unigram language model.
        try:
            # Convert 'words' to a list to ensure we can access elements by index.
            words = list(words)
            # Calculate the probability of the sequence by taking the product of individual word probabilities.
            prob = np.prod(self.mdl.loc[words])
        except KeyError:
            # If any word in the sequence is not present in the language model's vocabulary,
            # set the probability to 0 indicating that the sequence is not recognized.
            prob = 0
        return prob
        
    def sample(self, M):
        # Sampling method
        # Generates a random sequence of 'M' words from the language model.
        # 'M' is the number of words to be sampled.
        return ' '.join(self.mdl.sample(M, replace=True).index)
        # Generate a random sample of 'M' words with replacement from the language model's vocabulary.
        # Join the sampled words into a string with spaces between them and return the string.


### ✅ Testing: UnigramML() on The Great Gatsby

In [62]:
tokens = tokenized_gatsby

# Initilize and train Unifrom model on given tokens
unif = UnigramLM(tokens)

# Generates a random sequence of 100 words from the language model.
unif.sample(100)

'chill bug bumped lie spotted sleeves astounded actress fenders dull shots lots Ahead imagine America telephoned crickets unable Now invitations attempting fantastic cheekbone sleep cornices toiled inventions finished caf humming further capable burns moving generations grocery shall charm Another suite workmen happens state vague centre tangle saloons always roaring milky kinds Goddard flood rounds silence sports embroidered date dispute inessential sixteen subdued tag dissimilarity period distaste incident July determinedly substitute sold alongside raining reserve marvellous Bay dates declaration Dumbell signed roads carrying imagination bug cr hollow groceries breathlessly countered postern purposeless cleaned a Hulking policeman mean inquisitions border group spidery'

## 🎳 Creating NGram Model

The N-Gram language model relies on the assumption that only nearby tokens matter. Specifically, it assumes that the probability that a token occurs depends only on the previous $N-1$ tokens, rather than all previous tokens. That is:

$$P(w_n|w_1,\ldots,w_{n-1}) = P(w_n|w_{n-(N-1)},\ldots,w_{n-1})$$

In an N-Gram language model, there is a hyperparameter that we get to choose when creating the model, $N$. For any $N$, the resulting N-Gram model looks at the previous $N-1$ tokens when computing probabilities. (Note that the unigram model you built in Question 4 is really an N-Gram model with $N=1$, since it looked at 0 previous tokens when computing probabilities.)

<br>

#### Example: Trigram Model

When $N=3$, it's a "trigram" model. Such a model looks at the previous $N-1 = 2$ tokens when computing probabilities.

Consider the tuple `('when', 'I', 'drink', 'Coke', 'I', 'smile')`, corresponding to the sentence `'when I drink Coke I smile'`. Under the trigram model, the probability of this sentence is computed as follows:

$$P(\text{when I drink Coke I smile}) = P(\text{when}) \cdot P(\text{I | when}) \cdot P(\text{drink | when I}) \cdot P(\text{Coke | I drink}) \cdot P(\text{I | drink Coke}) \cdot P(\text{smile | Coke I})$$

The trigram model doesn't consider the beginning of the sentence when computing the probability that the sentence ends in `'smile'`.

#### N-Grams

Both when working with a training corpus and when implementing the `probability` method to compute the probabilities of other sentences, I use  "chunks" of $N$ tokens at a time.

**Definition:** The **N-Grams of a text** are a list of tuples containing sliding windows of length $N$.

For instance, the trigrams in the sentence `'when I drink Coke I smile'` are:

```py
[('when', 'I', 'drink'), ('I', 'drink', 'Coke'), ('drink', 'Coke', 'I'), ('Coke', 'I', 'smile')]
```

<br>

#### Computing N-Gram Probabilities

Notice in our trigram model above, I computed $P(\text{when I drink Coke I smile})$ as being the product of several conditional probabilities. These conditional probabilities are the result of **training** our N-Gram model on a training corpus.

To train an N-Gram model, I compute a conditional probability for every $N$-token sequence in the corpus. For instance, for every 3-token sequence $w_1, w_2, w_3$, I must compute $P(w_3 | w_1, w_2)$. To do so, I use:

$$P(w_3 | w_1, w_2) = \frac{C(w_1, w_2, w_3)}{C(w_1, w_2)}$$

where $C(w_1, w_2, w_3)$ is the number of occurrences of the trigram sequence $w_1, w_2, w_3$ in the training corpus and $C(w_1, w_2)$ is the number of occurrences of the bigram sequence  $w_1, w_2$ in the training corpus. (Technical note: the probabilities that I compute using the ratios of counts are _estimates_ of the true conditional probabilities of N-Grams in the population of corpuses from which our corpus was drawn.)

In general, for any $N$, conditional probabilities are computed by dividing the counts of N-Grams by the counts of the (N-1)-Grams they follow. 

<br>

### The `NGramLM` Class

The `NGramLM` class contains a few extra methods and attributes beyond those of `UniformLM` and `UnigramLM`:

1. Instantiating `NGramLM` requires both a list of tokens and a positive integer `N`, specifying the N in N-grams. This parameter is stored in an attribute `N`.
1. The `NGramLM` class has a method `create_ngrams` that takes in a list of tokens and returns a list of N-Grams (recall from above, an N-Gram is a **tuple** of length N). This list of N-Grams is then passed to the `train` method to train the N-Gram model.
1. While the `train` method still creates a language model (in this case, an N-Gram model) and stores it in the `mdl` attribute, this model is most naturally stored as a DataFrame. This DataFrame will have three columns:
    - `'ngram'`, containing the N-Grams found in the text.
    - `'n1gram'`, containing the (N-1)-Grams upon which the N-Grams in `ngram` are built.
    - `'prob'`, containing the probabilities of each N-Gram in `ngram`.
1. The `NGramLM` class has an attribute `prev_mdl` that stores an (N-1)-Gram language model over the same corpus (which in turn will store an (N-2)-Gram language model over the same corpus, and so on). This is necessary to compute the probability that a word occurs at the start of a text. 

N-Gram LM consists of probabilities of the form

$$P(w_n|w_{n-(N-1)},\ldots,w_{n-1})$$

Which can be estimated by:  

$$\frac{C(w_{n-(N-1)}, w_{n-(N-2)}, \ldots, w_{n-1}, w_n)}{C(w_{n-(N-1)}, w_{n-(N-2)}, \ldots, w_{n-1})}$$

for every N-Gram that occurs in the corpus. To illustrate, consider again the following example corpus:

```py
>>> corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
>>> tokens = tokenize(corpus)
>>> tokens
['\x02', 'when', 'I', 'eat', 'pizza', ',', 'I', 'smile', ',', 'but', 'when', 'I', 'drink', 'Coke', ',', 'my', 'stomach', 'hurts', '\x03']
>>> pizza_model = NGrams(3, tokens)
```

Here, `pizza_model.train` must compute $P(\text{I | \x02 when})$, $P(\text{eat | when I})$, $P(\text{pizza | I eat})$, and so on, until $P(\text{\x03 | stomach hurts})$.

To compute $P(\text{eat | when I})$, I find the number of occurrences of `'when I eat'` in the training corpus, and divide it by the number of occurrences of `'when I'` in the training corpus. `'when I eat'` occurred exactly once in the training corpus, while `'when I'` occurred twice, so,

$$P(\text{eat | when I}) = \frac{C(\text{when I eat})}{C(\text{when I})} = \frac{1}{2}$$

To store the conditional probabilities of all N-Grams, I use a DataFrame with three columns, like so:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ngram</th>
      <th>n1gram</th>
      <th>prob</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>(when, I, drink)</td>
      <td>(when, I)</td>
      <td>0.5</td>
    </tr>
    <tr>
      <th>1</th>
      <td>(when, I, eat)</td>
      <td>(when, I)</td>
      <td>0.5</td>
    </tr>
    <tr>
      <th>2</th>
      <td>(,, but, when)</td>
      <td>(,, but)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>(,, I, smile)</td>
      <td>(,, I)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>(I, smile, ,)</td>
      <td>(I, smile)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>5</th>
      <td>(,, my, stomach)</td>
      <td>(,, my)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>6</th>
      <td>(but, when, I)</td>
      <td>(but, when)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>7</th>
      <td>(, when, I)</td>
      <td>(, when)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>8</th>
      <td>(stomach, hurts, )</td>
      <td>(stomach, hurts)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>9</th>
      <td>(Coke, ,, my)</td>
      <td>(Coke, ,)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>10</th>
      <td>(eat, pizza, ,)</td>
      <td>(eat, pizza)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>11</th>
      <td>(I, drink, Coke)</td>
      <td>(I, drink)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>12</th>
      <td>(my, stomach, hurts)</td>
      <td>(my, stomach)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>13</th>
      <td>(pizza, ,, I)</td>
      <td>(pizza, ,)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>14</th>
      <td>(I, eat, pizza)</td>
      <td>(I, eat)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>15</th>
      <td>(drink, Coke, ,)</td>
      <td>(drink, Coke)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>16</th>
      <td>(smile, ,, but)</td>
      <td>(smile, ,)</td>
      <td>1.0</td>
    </tr>
  </tbody>
</table>

The row at position **1** in the above table shows that the probability of the trigram `('when', 'I', 'eat')` conditioned on the bigram `('when', 'I')` is 0.5, as we computed above. Note that many of the above conditional probabilities are equal to 1 because many trigrams and their corresponding bigrams each appeared only once, and $\frac{1}{1} = 1$. Note that `'\x02'` and `'\x03'` appear as spaces above, such as in row **7**.


In [76]:
class NGramLM(object):
    
    def __init__(self, N, tokens):
        # Constructor method
        # Initializes the class instance for an N-gram language model.
        # 'N' is the order of the N-gram model.
        # 'tokens' is a list of words used to train the language model.
        
        self.N = N
        # Store the N-gram order in the class attribute 'N'.
        
        ngrams = self.create_ngrams(tokens)
        self.tok = tokens
        self.ngrams = ngrams
        self.mdl = self.train(ngrams)
        # Create N-grams from the provided tokens and train the N-gram language model.
        
        if N < 2:
            raise Exception('N must be greater than 1')
        # Check if N is less than 2, which means the model is not valid (N should be 2 or greater).
        
        elif N == 2:
            self.prev_mdl = UnigramLM(tokens)
        else:
            self.prev_mdl = NGramLM(N-1, tokens)
        # If N is 2, create a UnigramLM as a previous model (prev_mdl).
        # Otherwise, create an (N-1)-gram language model as a previous model (prev_mdl).
        
    def create_ngrams(self, tokens):
        # N-gram creation method
        # Creates N-grams from a list of tokens.
        # 'tokens' is a list of words used to build the N-grams.
        
        result = []
        right_pointer = self.N
        left_pointer = 0
        
        while right_pointer <= len(tokens):
            # Create N-gram by taking a slice of tokens from left_pointer to right_pointer.
            result.append(tuple(tokens[left_pointer:right_pointer]))
            left_pointer += 1
            right_pointer += 1
        
        return result
        # Return the list of N-grams created from the tokens.
    
    def create_ngrams_N(self, n, tokens):
        # N-gram creation method for a specific value of N
        # Creates N-grams from a list of tokens.
        # 'n' is the specific N-gram order.
        # 'tokens' is a list of words used to build the N-grams.
        
        res = []
        right = n
        left = 0

        while right <= len(tokens):
            # Create N-gram by taking a slice of tokens from left to right.
            res.append(tuple(tokens[left:right]))
            left += 1
            right += 1
        
        return res
        # Return the list of N-grams created from the tokens.
        
    def train(self, ngrams):
        # N-gram training method
        # Calculates probabilities for each N-gram in the training data.
        # 'ngrams' is the list of N-grams used to train the model.
        
        # Create pandas Series from N-grams to facilitate computations.
        ser_ngram = pd.Series(ngrams, name="ngram")
        # Create (N-1)-grams from the tokens.
        n1gram_create = self.create_ngrams_N(self.N-1, self.tok)
        ser_n1gram = pd.Series(n1gram_create, name="n1gram")

        # Merge N-grams and (N-1)-grams on common elements to calculate probabilities.
        merge_ngram_n1gram = pd.merge(ser_ngram, ser_n1gram, left_index=True, right_index=True)
        # Calculate counts of each (N-1)-gram to use as denominators for probabilities.
        denom_col = merge_ngram_n1gram.groupby("n1gram").transform('count')
        # Calculate counts of each N-gram to use as numerators for probabilities.
        numer_col = merge_ngram_n1gram.groupby("ngram").transform('count')
        # Add numerator and denominator columns to the DataFrame.
        merge_ngram_n1gram["numerator"] = numer_col
        merge_ngram_n1gram["denominator"] = denom_col
        # Calculate probabilities for each N-gram based on the counts.
        merge_ngram_n1gram["prob"] = merge_ngram_n1gram["numerator"] / merge_ngram_n1gram["denominator"]
        
        # Drop the intermediate columns and keep only unique N-grams with their probabilities.
        return merge_ngram_n1gram.drop(columns=["numerator", "denominator"]).drop_duplicates(keep='first')
    
    def probability(self, words):
        # Probability calculation method
        # It calculates the probability of a sequence of words given the trained N-gram language model.
        # 'words' is the sequence of words for which the probability is calculated.
        
        input_words = ' '.join(words)
        token_words = ' '.join(self.tok)

        if input_words not in token_words:
            # If the input sequence is not present in the training data, return 0 probability.
            return 0
        
        final_prob = 1
        
        initial_prob = []
        for i in range(self.N-1, 0, -1):
            if i == 1:
                initial_prob.append(words[0:i][0])
            else:
                initial_prob.append(words[0:i])
        # Create a list of (N-1)-grams that precede the input word sequence.

        list_grams = self.create_ngrams(words)
        # Create N-grams from the input word sequence.
        
        for gram in list_grams: 
            if gram not in list(self.mdl['ngram'].values):
                # If any N-gram in the input word sequence is not present in the model, return 0 probability.
                return 0
            else:
                final_prob *= self.mdl[self.mdl['ngram'] == gram]['prob'].iloc[0]
                # Multiply the probabilities of each N-gram in the input word sequence.
                # Access the corresponding probability from the trained N-gram model.

        current = self
        for gram in initial_prob:
            # Loop through the list of (N-1)-grams in reverse order (from largest to smallest).
            current = current.prev_mdl
            # Traverse to the previous model for each (N-1)-gram.
            current_table = current.mdl
            # Access the probability table of the previous model.

            if isinstance(gram, str):
                # If the (N-1)-gram is a single word (unigram), access its probability from the unigram model.
                final_prob *= pd.DataFrame(current_table).loc[gram].iloc[0]
            else:
                # If the (N-1)-gram is not a unigram, access its probability from the N-1 gram model.
                final_prob *= current_table[current_table["ngram"] == gram]["prob"].iloc[0]
        # Multiply probabilities of (N-1)-grams that precede the input word sequence.

        return final_prob
        # Return the final calculated probability of the input word sequence.
        
    def sample(self, M):
        # Sampling method
        # It generates a random sequence of 'M' words from the language model.
        # 'M' is the number of words to be sampled.
        
        sample_words = ['\x02']
        # Initialize the list of sampled words with a special start symbol '\x02'.
        
        for i in range(1, self.N):
            # Loop from 1 to N-1 to generate initial (N-1)-grams.
            current = self.N - i
            prev = self
            
            for i in range(current, 1, -1):
                # Traverse to the previous model for each (N-1)-gram.
                prev = prev.prev_mdl
            
            # Access the probability table for the (N-1)-gram model.
            probability = prev.mdl[np.where(prev.mdl['n1gram'].apply(str) == str(tuple(sample_words)), True, False)].drop_duplicates(keep='first')
            
            if probability.shape[0] == 0 or probability.shape[1] == 0:
                sample_words.append('\x03')
                # If no probabilities found for the (N-1)-gram, add the end symbol '\x03'.
            else:
                # Sample the next word using the probabilities from the (N-1)-gram model.
                smalls = np.random.choice(probability['ngram'], p=probability['prob'])
                sample_words.append(smalls[-1])
                # Append the last word of the sampled (N-1)-gram to the sampled words list.
        
        for j in range(self.N, M+1):
            # Loop from N to M (inclusive) to generate N-grams for the rest of the sequence.
            probability = prev.mdl[np.where(prev.mdl['n1gram'].apply(str) == str(tuple(sample_words[-self.N+1:])), True, False)].drop_duplicates(keep='first')
            
            if probability.shape[0] == 0 or probability.shape[1] == 0:
                sample_words.append('\x03')
                # If no probabilities found for the N-gram, add the end symbol '\x03'.
            else:
                # Sample the next word using the probabilities from the N-gram model.
                words = np.random.choice(probability['ngram'], p=probability['prob'])
                sample_words.append(np.random.choice(probability['ngram'], p=probability['prob'])[-1])
                # Append the last word of the sampled N-gram to the sampled words list.
        
        if sample_words[-1] != '\x03':
            sample_words[-1] = '\x03'
            # Replace the last word with the end symbol '\x03'.
        
        return ' '.join(sample_words)
        # Join the sampled words into a string with spaces between them and return the string.


### ✅ Testing: N() on The Great Gatsby

In [98]:
ngram = NGramLM(3, tokenized_gatsby)
ngram_series_representation = ngram.create_ngrams(tokens)
ngram_dataframe_representation = ngram.mdl
ngram_model = ngram

In [99]:
# Generate sample sentence built from probability table
ngram_sample = ngram.sample(500) 
ngram_sample

'\x02 “ And your house some afternoon and Long Distance said Chicago was calling up a nice restaurant here , ” mumbled Miss Baedeker ? ” \x03 \x02 Cody was fifty years old , ” said Wilson , nodding into the twilight , but a whole floor of the youth and , even through his punctilious manner in the room . With Jordan ’ s a regular Belasco . It ’ s voice calling a taxi go up Gatsby ’ s hospitality and so it came about that in a low husky sob , and Doctor Webster Civet , who had arrived , they were selling something : bonds or insurance or automobiles . They ’ re Jordan Baker . ( “ Rot - Gut ” ) Ferret and the adventitious authority of his destiny , to be brought to him , but Wilson wouldn ’ t reached West Egg . ” \x03 \x02 “ You don ’ t get mixed up in his car . ” \x03 \x02 “ Does the gasoline affect his nose ? ” asked Mrs . McKee , “ but full of cheerful sun . Sitting down behind the tall apartments of the sofa and his mistress , until , as we drove away . \x03 \x02 It was the man smokin